!pip install census

!pip install us

In [1]:
from census import Census
from us import states
import pandas as pd
import numpy as np
import pylab as pl
import csv
%pylab inline

Populating the interactive namespace from numpy and matplotlib


#### Insert your census api key here below

In [2]:
c = Census("4c26aa6ebbaef54a55d3903212eabbb506ade381")

#### This is the data dictionary

#### Creating the buckets for variables that are separeted into bins like income

In [3]:
buckets = [0, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 
           60000, 75000, 100000, 125000, 150000, 200000,1000000]

In [4]:
buckets_2 = [0, 10000, 15000, 25000, 35000, 50000, 65000, 75000, 1000000]

#### Insert the variables you want to download. The second part is just so it is automated.

In [5]:
variables = ('B01003', 'B19001', 'B15003', 'B25003', 'B02001', 'B19013', 'B25003', 'B25064', 'B25077', 'B25034')

In [6]:
var=[]
for i in variables:
    var.append(i+'_001E')
    var.append(i+'_001M')
    if i=='B02001':
        var.append(i+'_002E')
        var.append(i+'_002M')
    if i=='B25003':
        var.append(i+'_002E')
        var.append(i+'_002M')
        var.append(i+'_003E')
        var.append(i+'_003M')
    if i=='B25034':
        var.append(i+'_010E')
        var.append(i+'_010M')
        var.append(i+'_011E')
        var.append(i+'_011M')
    #var.append(i+'_001MA')
    #var.append(i+'_001EA')

In [7]:
var.append('NAME')
for i in range(2,10):
    var.append('B19001_00'+str(i)+'E')
    var.append('B19001_00'+str(i)+'M')
for i in range(10,18):
    var.append('B19001_0'+str(i)+'E')
    var.append('B19001_0'+str(i)+'M')
for i in range(3,10):
    var.append('B07010_00'+str(i)+'E')
    var.append('B07010_00'+str(i)+'M')
for i in range(10,12):
    var.append('B07010_0'+str(i)+'E')
    var.append('B07010_0'+str(i)+'M')

In [8]:
var.append('B25034_001E')
var.append('B25034_001M')
var.append('B25034_010E')
var.append('B25034_010M')
var.append('B25034_011E')
var.append('B25034_011M')

#### Choose the state and insert it as part as the api json

In [9]:
state = '09'

In [10]:
sql_query='state:{} county:*'.format(state)

In [11]:
dic = c.acs5.get(var, geo = {'for': 'tract:*',
                       'in': sql_query})

In [12]:
df = pd.DataFrame.from_dict(dic)

In [13]:
df = df.reset_index()

In [14]:
df = df.drop(columns = 'index')

#### Exporting the raw file

In [15]:
df.to_csv('variables_' + state + '.csv', index = None)

#### Calculating the thresholds for each of the counties

In [16]:
county = df.county.unique()

In [17]:
counties = {}
counties = counties.fromkeys(county)

In [18]:
for i in county:
    counties[i] = np.median(df[df.county == i].B19013_001E)

In [19]:
df_threshold = pd.DataFrame.from_dict(counties, orient='index')

In [20]:
df_threshold = df_threshold.rename(columns={0:'median'})

In [21]:
#df_typo['vvli_threshold']=0.3*df_typo['median']
df_threshold['vli_threshold'] = 0.5*df_threshold['median']
df_threshold['li_threshold'] = 0.8*df_threshold['median']
df_threshold['mi_threshold'] = 1.2*df_threshold['median']
df_threshold['mhi_threshold'] = 1.5*df_threshold['median']
df_threshold['hi_threshold'] = 2*df_threshold['median']
df_threshold['vhi_threshold'] = 1000000

#### Creating the different typologies based on each county's median income

In [22]:
table = np.zeros((len(county),len(buckets)-1))
for j in range(0, len(counties)):
    col = 1
    for i in range(0, len(buckets)-1):
        if buckets[i+1] < df_threshold.iloc[j, col]:
            table[j, i] = True
        else:
            table[j, i] = (df_threshold.iloc[j, col] - buckets[i]) / (buckets[i+1] - buckets[i])
            col = col+1

In [23]:
values=[]
for i in county:
    with open('variables_'+state+'.csv', 'r') as fi:
        reader = csv.DictReader(fi)
        def creator(reader, c):
            bt = filter(lambda x: x['county'] == c, reader)
            return (list(bt))
        value = creator(reader, i)
    values.append(value)

In [24]:
income_typo = []
for j in range(0, len(county)):
    for i in range(0, len(values[j])):
        typo = []
        summ = 0
        for col in range(0,len(buckets)-1):
            if ((table[j, col] == 1) & (values[j][i][var[2]]!='0.0')):
                summ = summ + float(values[j][i][var[35+col*2]])/float(values[j][i][var[2]])
            else:
                if values[j][i][var[2]]!='0.0':
                    summ = summ + float(values[j][i][var[35+col*2]])*table[j, col]/float(values[j][i][var[2]])
                    typo.append(summ)
                    summ = float(values[j][i][var[35+col*2]])*(1-table[j, col])/float(values[j][i][var[2]])
        typo.append(summ)
        income_typo.append(typo)

In [25]:
df_final = pd.DataFrame(income_typo)

#### Calculating the margin of error for each typology

In [26]:
income_moe = []
for j in range(0, len(county)):
    for i in range(0, len(values[j])):
        typo = []
        summ = 0
        for col in range(0,len(buckets)-1):
            dividend_1 = float(values[j][i][var[36+col*2]])
            divisor_1 = float(values[j][i][var[35+col*2]])
            dividend_2 = float(values[j][i][var[3]])
            divisor_2 = float(values[j][i][var[2]])
            if ((divisor_1 != 0) & (divisor_2 != 0)):
                error = (dividend_1/divisor_1 + dividend_2/divisor_2)*(divisor_1/divisor_2)
            else:
                error = 0
            if (table[j, col] == 1):
                summ = summ + error
            else:
                summ = summ + error*table[j, col]
                typo.append(summ)
                summ = (1-table[j, col])*error
        typo.append(summ)
        income_moe.append(typo)

In [27]:
income_moe_df = pd.DataFrame(income_moe)

In [28]:
income_moe_df = income_moe_df.rename(columns={0:'vli_16_moe', 
                                              1:'li_16_moe', 
                                              2:'mi_16_moe', 3:'mhi_16_moe', 4:'hi_16_moe', 5:'vhi_16_moe'})

In [29]:
df_final = df_final.rename(columns={0:'vli_16', 1:'li_16', 2:'mi_16', 3:'mhi_16', 4:'hi_16', 5:'vhi_16'})

#### Creating the migration dataframe

In [30]:
table_2 = np.zeros((len(county),len(buckets_2)-1))
for j in range(0, len(county)):
    col = 1
    for i in range(0, len(buckets_2)-1):
        if buckets_2[i] > df_threshold.iloc[j, col]:
            table_2[j, i] = 0
            col = col+1
        elif buckets_2[i+1] < df_threshold.iloc[j, col]:
                table_2[j, i] = True
        else:
            table_2[j, i] = (df_threshold.iloc[j, col] - buckets_2[i]) / (buckets_2[i+1] - buckets_2[i])
            col = col+1

In [31]:
census_tract = []
for j in range(0, len(county)):
    for i in range(0, len(values[j])):
        typo = []
        summ = 0
        for col in range(0,len(buckets_2)-1):
            moved = float(values[j][i][var[69+col*2]])
            if float(values[j][i][var[67]])!=0:
                per = moved/float(values[j][i][var[67]])
            else:
                per = 0
            if table_2[j, col] == 1:
                summ = summ + per
            elif table_2[j, col] != 0:
                summ = summ + per*table_2[j, col]
                typo.append(summ)
                summ = per*(1-table_2[j, col])
            elif ((table_2[j, col] == 0)&(float(values[j][i][var[67]])!=0)):
                typo.append(summ)
                col = col+1
                summ = per * table_2[j, col] + float(values[j][i][var[69+(col-1)*2]])/float(values[j][i][var[67]]) 
        typo.append(summ)
        census_tract.append(typo)

#### Calculating the margin of error for the migration typologies

In [32]:
mig_moe = []
for j in range(0, len(county)):
    for i in range(0, len(values[j])):
        typo = []
        summ = 0
        for col in range(0,len(buckets_2)-1):
            dividend_1 = float(values[j][i][var[70+col*2]])
            divisor_1 = float(values[j][i][var[69+col*2]])
            dividend_2 = float(values[j][i][var[68]])
            divisor_2 = float(values[j][i][var[67]])
            dividend = float(values[j][i][var[70+(col+1)*2]])
            divisor = float(values[j][i][var[69+(col+1)*2]])
            if ((divisor_1 != 0) & (divisor_2 != 0)):
                error = (dividend_1/divisor_1 + dividend_2/divisor_2)*(divisor_1/divisor_2)
            else:
                error = 0
            if (table_2[j, col] == 1):
                summ = summ + error
            elif ((table_2[j, col] == 0)&(((divisor != 0) & (divisor_2 != 0)))):
                error_2 = (dividend/divisor + dividend_2/divisor_2)*(divisor/divisor_2)
                typo.append(summ)
                col = col+1
                summ = summ + error_2*table_2[j, col] + error*table_2[j, col-1]
            else:
                summ = summ + error*table_2[j, col]
                typo.append(summ)
                summ = (1-table_2[j, col])*error
        typo.append(summ)
        mig_moe.append(typo)

In [33]:
mig_moe_df = pd.DataFrame(mig_moe)

In [34]:
mig_moe_df = pd.DataFrame(np.nan_to_num(mig_moe_df))

In [35]:
df_mig = pd.DataFrame(census_tract)

In [36]:
df_mig = pd.DataFrame(np.nan_to_num(df_mig))

In [37]:
df_mig = df_mig.rename(columns={0:'vli_mig_16', 1:'li_mig_16', 2:'mi_mig_16', 3:'mhi_mig_16', 4:'hi_mig_16', 
                                5:'vhi_mig_16'})
mig_moe_df = mig_moe_df.rename(columns={0:'vli_mig_16_moe', 1:'li_mig_16_moe', 2:'mi_mig_16_moe', 3:'mhi_mig_16_moe',
                                        4:'hi_mig_16_moe',5:'vhi_mig_16_moe'})

In [38]:
df_final = pd.concat([df_final, df_mig, income_moe_df, mig_moe_df], axis=1)

In [39]:
df_final['per_all_li_mig_16'] = df_final['vli_mig_16'] + df_final['li_mig_16']

In [40]:
df_final['per_units_pre50'] = (df['B25034_010E'] + df['B25034_011E'])/df['B25034_001E']

In [41]:
moe_names = ['per_col_16', 'per_rent_16', 'per_owners_16', 'per_nonwhite_16']
moe_dividend = ['B15003_001', 'B25003_003', 'B25003_002', 'B02001_002']
moe_divisor = ['B01003_001', 'B25003_001', 'B25003_001', 'B01003_001']

#### Calculating the margin of error for variables that represent percentages

In [42]:
for i in range(0,len(moe_names)):
    if i!=3:
        df_final[moe_names[i]] = df[moe_dividend[i]+'E']/df[moe_divisor[i]+'E']
    else:
        df_final[moe_names[i]] = 1 - df[moe_dividend[i]+'E']/df[moe_divisor[i]+'E']
    df_final[moe_names[i]+'_moe'] = df_final[moe_names[i]].multiply(
                               (df[moe_dividend[i]+'M'].divide(df[moe_dividend[i]+'E'], axis = 'index')+
                                df[moe_divisor[i]+'M'].divide(df[moe_divisor[i]+'E'], axis = 'index')), axis = 'index')

In [43]:
df = df.rename(columns={'B19001_001E':'hh_16', 'B25064_001E':'mrent_16', 'B19013_001E':'hinc_16', 
                        'B25077_001E':'mhval_16', 'B25003_001E':'hu_16', 'B01003_001E':'pop_16',
                        'B19001_001M':'hh_16_moe', 'B25064_001M':'mrent_16_moe', 'B19013_001M':'hinc_16_moe', 
                        'B25077_001M':'mhval_16_moe', 'B25003_001M':'hu_16_moe', 'B01003_001M':'pop_16_moe'})

In [44]:
df_final = df_final.join(df[['hh_16', 'mrent_16', 'hinc_16', 'mhval_16', 'hu_16', 'pop_16', 'NAME', 'state', 'county',
                             'tract', 'hh_16_moe', 'mrent_16_moe', 'hinc_16_moe', 'mhval_16_moe', 'hu_16_moe', 
                             'pop_16_moe']])

#### Droping null values

In [45]:
prev_len = len(df_final)

In [46]:
df_final = df_final.dropna(axis = 0, how = 'any')

In [47]:
final_len = len(df_final)

In [48]:
print('The percentage of information lost is: {}'.format((prev_len-final_len)*100/prev_len))

The percentage of information lost is: 1.440576230492197


#### Exporting the final file

In [49]:
df_final['GEOid2'] = df_final['state'] + df_final['county'] + df_final['tract']

In [50]:
df_final.to_csv(state + '_2016_variables.csv', index = None)